<a href="https://colab.research.google.com/github/satvikk/ai_synthesize/blob/master/learn0_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN on MNIST

In [0]:
import torch as t
import plotly.graph_objects as go
import torchvision.datasets as datasets
t.set_default_tensor_type(t.cuda.FloatTensor)
import torch.nn.functional as F
import tqdm

In [0]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [0]:
grid20x = t.cat([t.linspace(0,27,28)]*28)
grid20y = t.cat([t.linspace(27,0,28).unsqueeze(1)]*28, dim = 1).flatten()
fig = go.Figure()
fig.add_scatter(
    x = grid20x.cpu(),
    y = grid20y.cpu(),
    mode = "markers",
    marker = dict(
        color = mnist_trainset.data[56550,:,:].flatten().cpu(),
        symbol = "square",
        size = 15,
    )
)
fig.update_layout(
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()

In [0]:
class Gigi0(t.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv0 = t.nn.Conv2d(1,1, kernel_size=(5,5), padding = 2)
    self.conv1 = t.nn.Conv2d(1,1, kernel_size=(5,5), padding = 2)
    self.lin0 = t.nn.Linear(7*7, 128)
    self.lin1 = t.nn.Linear(128, 10)
    self.dropout0 = t.nn.Dropout(0.0)
    self.dropout1 = t.nn.Dropout(0.4)

  def forward(self, x):
    x = x.unsqueeze(1)
    x = self.dropout0(F.relu(self.conv0(x)))
    x = F.max_pool2d(x, 2)
    x = self.dropout0(F.relu(self.conv1(x)))
    x = F.max_pool2d(x, 2)
    x = x.reshape(-1,1,7*7)
    x = self.dropout1(F.relu(self.lin0(x)))
    x = self.dropout1(self.lin1(x))
    x = F.log_softmax(x, 2).squeeze(1)
    return x

class Gigi1(t.nn.Module):
  def __init__(self):
    super().__init__()
    self.lin0 = t.nn.Linear(28*28, 300)
    self.lin1 = t.nn.Linear(300, 100)
    self.lin2 = t.nn.Linear(100, 10)
    self.dropout = t.nn.Dropout(0.4)

  def forward(self, x):
    x = x.reshape(-1,1,28*28)
    x = self.dropout(F.relu(self.lin0(x)))
    x = self.dropout(F.relu(self.lin1(x)))
    x = self.dropout(self.lin2(x))
    x = F.log_softmax(x, 2).squeeze(1)
    return x

model = Gigi0()

In [0]:
in_data = mnist_trainset.data[342:349,:,:].unsqueeze(1).float().cuda()
conv0 = t.nn.Conv2d(1,1, kernel_size=(5,5), padding = 2)
conv1 = t.nn.Conv2d(1,1, kernel_size=(5,5), padding = 2)
out0 = F.relu(conv0(in_data))
out1 = F.max_pool2d(out0, 2)
out2 = conv1(out1)
out3 = F.max_pool2d(out2, 2)
# lin0 = t.nn.Linear(36, 5)
# out3 = F.log_softmax(lin0(out2), 2).squeeze(1)
# loss = F.nll_loss(out3, t.tensor([0,4,1,3,1,2,0]).long())

In [0]:
print(in_data.shape)
print(out0.shape)
print(out1.shape)
print(out2.shape)
print(out3.shape)

torch.Size([7, 1, 28, 28])
torch.Size([7, 1, 28, 28])
torch.Size([7, 1, 14, 14])
torch.Size([7, 1, 14, 14])
torch.Size([7, 1, 7, 7])


In [0]:
class datamaker(t.utils.data.Dataset):
    def __init__(self, mnist):
        self.mnist = mnist   

    def __len__(self):
        return len(self.mnist)
    
    def __getitem__(self, idx):
        return {'x': self.mnist.data[idx].cuda().float(), 'y': self.mnist.targets[idx].cuda().long()}

dataloader = t.utils.data.DataLoader(datamaker(mnist_trainset), batch_size=100,shuffle=True,)

In [0]:
n_epochs = 20
init_lr = 0.001
final_lr = 1.
lr_rate = (final_lr - init_lr)/n_epochs

model.train()
ema_loss = 2.4
optimizer = t.optim.Adam(model.parameters(), lr = init_lr, betas=(0.9,0.999))
for epochs in range(n_epochs):
  for dat in (dataloader):
    # if i > 6:
      # break;
    # print(model(dat['x']))
    optimizer.zero_grad()
    output = model(dat['x'])
    loss = F.nll_loss(output, dat['y'])
    loss.backward()
    optimizer.step()
    ema_loss = ema_loss*0.99 + loss.item()*0.01
  # optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] + lr_rate
  with t.no_grad():
    # b = model(mnist_trainset.data.cuda().float()).argmax(1)
    # accuracy_train = t.sum(mnist_trainset.targets.cuda() == b).float()/b.shape[0]
    b = model(mnist_testset.data.cuda().float()).argmax(1)
    accuracy_test = t.sum(mnist_testset.targets.cuda() == b).float()/b.shape[0]
  print(str(epochs) + "/" + str(n_epochs)+ ": "+str(ema_loss)+" | " + str(accuracy_test.item()*100)+"%")

0/20: 0.8465711228650097 | 64.95999693870544%
1/20: 0.8280476728456365 | 65.27000069618225%
2/20: 0.821741175280645 | 65.32999873161316%
3/20: 0.8139403291500267 | 65.68999886512756%
4/20: 0.8143255141252516 | 65.1699960231781%
5/20: 0.8002829084438309 | 64.52999711036682%
6/20: 0.7917381146925776 | 66.25999808311462%
7/20: 0.7875690924930248 | 66.1899983882904%
8/20: 0.8033710918046221 | 65.34000039100647%
9/20: 0.7817038419370972 | 65.75999855995178%
10/20: 0.7817250443227005 | 64.82999920845032%
11/20: 0.7788845526189918 | 66.32999777793884%
12/20: 0.7925627518461198 | 66.13999605178833%
13/20: 0.7770271971144733 | 65.66999554634094%
14/20: 0.7809690644745898 | 65.32999873161316%
15/20: 0.78179441425122 | 66.1799967288971%
16/20: 0.7838379307074027 | 65.56999683380127%
17/20: 0.7782228129966584 | 65.78999757766724%
18/20: 0.771767179145124 | 65.70000052452087%
19/20: 0.7689972303458329 | 65.95999598503113%


In [0]:
model.eval()
with t.no_grad():
    b = model(mnist_testset.data.cuda().float()).argmax(1)
    accuracy = t.sum(mnist_testset.targets.cuda() == b).float()/b.shape[0]
print(accuracy.item()*100)

95.90999484062195


In [0]:
# layer = model.conv1.weight[0][0,:,:].detach()
irow = 56550
layer = mnist_trainset.data[irow:(irow+1),:,:].unsqueeze(1).cuda().float()
layer = F.relu(model.conv0(layer))
layer = F.max_pool2d(layer, 2)
layer = F.relu(model.conv1(layer)).detach()
size = 14
grid20x = t.cat([t.linspace(0,size-1,size)]*size)
grid20y = t.cat([t.linspace(size-1,0,size).unsqueeze(1)]*size, dim = 1).flatten()
fig = go.Figure()
fig.add_scatter(
    x = grid20x.cpu(),
    y = grid20y.cpu(),
    mode = "markers",
    marker = dict(
        color = layer.flatten().cpu(),
        symbol = "square",
        size = 15,
    )
)
fig.update_layout(
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()
print(mnist_trainset.targets[irow])

tensor(4, device='cpu')
